In [49]:
import spacy
from spacy import displacy
from collections import Counter, defaultdict
import en_core_web_sm
nlp = en_core_web_sm.load()
import numpy as np
from openie import StanfordOpenIE
import wptools
import wikipedia
import pickle
from tqdm import tqdm
import regex as re
import mwclient
import warnings
warnings.filterwarnings('ignore')

In [98]:
# with StanfordOpenIE() as client:
#     text = page_summary
#     tmp = client.annotate(text)  

In [92]:
# subjects = {}
# for idx,triple in enumerate(tmp):
# #     if(triple['subject'].lower() in list(lemma.keys())):
# #         subj = lemma[triple['subject'].lower()]
#     subj = triple['subject'].lower()
#     try:
#         _ = subjects[subj]
#     except:
#         subjects[subj] = defaultdict(list)
#     (subjects[subj][triple['relation'].lower()]).append(triple['object'].lower())
# #         print(subjects[subj][triple['relation'].lower()])
# #         print(triple['relation'].lower())
# #             print('|-', triple)

In [59]:
# entity_id = 'Q13897247'
# titles = wptools.page(wikibase = entity_id, silent = True).get_wikidata()
# title_list = titles.data['title'].split('_')
# title = " ".join(title_list)
# page_summary = wikipedia.summary(title)

In [99]:
# page_summary

In [14]:
def baseline(item):
    
    try:
        titles = wptools.page(wikibase = item, silent = True).get_wikidata()
        title_list = titles.data['title'].split('_')
        title = " ".join(title_list)
        page_summary = wikipedia.summary(title)
    except:
        return False
    
    try:
        with StanfordOpenIE() as client:
            text = page_summary
            tmp = client.annotate(text)
    except:
        return False
    
    subjects = {}
    for idx,triple in enumerate(tmp):
        subj = triple['subject'].lower()
        try:
            _ = subjects[subj]
        except:
            subjects[subj] = defaultdict(list)
        (subjects[subj][triple['relation'].lower()]).append(triple['object'].lower())
    
    relation_threshold = 3
#     object_threshold = 3

    subjs_dict = {}
    results = defaultdict(list)
    for key in subjects:
        relations_dict = {}
        words_list = (' '.join(list(subjects[key].keys()))).split(' ')
        words, word_cnts = np.unique(words_list, return_counts=True)
        relations_dict[key] = [(word, word_cnt) for word_cnt,word in sorted(zip(word_cnts,words),reverse=True)]

        thresh = len(relations_dict[key]) if len(relations_dict[key]) < relation_threshold else relation_threshold
        for word,_ in relations_dict[key][:thresh]:
            valid_rels = []
            for rels in list(subjects[key].keys()):
                if(word in rels and rels not in valid_rels):
                    valid_rels.append(rels)

        objects_list = []

        for rel in valid_rels: objects_list.extend(subjects[key][rel])

        objects_list = (' '.join(objects_list)).split(' ')
        objs, obj_cnts = np.unique(objects_list, return_counts=True)

        subjs_dict[key] = [(obj, obj_cnt) for obj, obj_cnt in sorted(zip(objs, obj_cnts),reverse=True)]


    #     thresh = len(subjs_dict[key]) if len(subjs_dict[key]) < object_threshold else object_threshold

        results[subjs_dict[key][0][0]].append(key)

    for res in results:
        results[res] = ', '.join(results[res])

    infobox = []
    for key,val in list(results.items()):
        infobox.append(key + ' = ' + val)
    
    return infobox, title

In [15]:
infobox_1,_ = baseline('Q13897247')

Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7444cdc8d0fc4ef6.props -preload openie


In [3]:
stats = defaultdict(list)

for i in range(1457):
    
    with open('./stats/' + 'instance_' + str(i) + '.pkl', 'rb') as f:
        data = pickle.load(f)
    
    for key in data:
        stats[key].extend(data[key])

with open('./updated_parsed_data.pkl','rb') as f:
    data = pickle.load(f)

dict_tmp = {}
for key in data: #data from parsed_data.pkl
    dict_tmp[data[key]['wikibase_item']] = data[key]['infobox']

for key in stats:
    for idx,item in enumerate(stats[key]):
        wikibase_item = item['item']
        if(wikibase_item in dict_tmp):
            stats[key][idx]['infobox'] = dict_tmp[wikibase_item]

In [4]:
total_count = 0
noinfo_count = 0
films_items_noinfobox = []
films_with_infobox = []
print('Instances considered :- ')
for key in stats:
    if('film' in key[-4:]):
        print(key)
        for val in stats[key]:
            if(val['infobox'] == 0):
                films_items_noinfobox.append(val['item'])
                noinfo_count += 1
            else:
                films_with_infobox.append(val['item'])
        total_count += len(stats[key])
print('\nTotal film pages: ',total_count)
print('Film no infobox: ',noinfo_count)

print('\n\n-------\n')

total_count = 0
noinfo_count = 0
sports_items_noinfobox = []
print('Instances considered :- ')
for key in stats:
    if('sport' in key[-5:] and 'transport' not in key):
        print(key)
        for val in stats[key]:
            if(val['infobox'] == 0):
                sports_items_noinfobox.append(val['item'])
                noinfo_count += 1
        total_count += len(stats[key])
print('\nTotal sports pages: ',total_count)
print('Sports no infobox: ',noinfo_count)

Instances considered :- 
film
short film
animated feature film
television film
feature film
silent short film
animated film
3D film
cult film
anthology film

Total film pages:  4872
Film no infobox:  1507


-------

Instances considered :- 
type of sport
sport
dog sport
fictional sport
individual sport
winter sport
mind sport

Total sports pages:  63
Sports no infobox:  45


In [5]:
films_with_infobox = films_with_infobox[:200]

In [6]:
films_infobox = {}
cnt = 0
for item in tqdm(films_with_infobox):
    tmp = baseline(item)
    if(tmp):
        films_infobox[item] = tmp
        cnt += 1
    print(cnt)

  0%|          | 0/200 [00:00<?, ?it/s]

Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-39be79560b4746ea.props -preload openie


  0%|          | 1/200 [00:25<1:25:06, 25.66s/it]

1
Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-2e7870df75104d6e.props -preload openie


  1%|          | 2/200 [00:48<1:22:09, 24.90s/it]

...

170
Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-d4c04803b2f64045.props -preload openie


100%|█████████▉| 199/200 [1:05:43<00:19, 19.77s/it]

171
Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-881597f145164726.props -preload openie


100%|██████████| 200/200 [1:06:03<00:00, 19.82s/it]

172


In [8]:
with open('film_infoboxes_baseline.pkl', 'wb') as handle:
    pickle.dump(films_infobox, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
def create_baseline_infobox_string(infobox, title):
    infobox_str = '{{Infobox ' + '\n'
    infobox_str += '| title = ' + title + '\n'
    cnt = 1
    for info in infobox:
        info = info.split('=')
        infobox_str += '|' + ' label' + str(cnt) + ' = ' + info[0].strip() + ' | ' + 'data' + str(cnt) + ' = ' + info[1].strip() + '\n'
        cnt += 1
    infobox_str += '}}'
    return infobox_str

In [56]:
infobox_1, title_1 = baseline('Q5890')

Starting server with command: java -Xmx8G -cp /home/aniket/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-f8f3500cb83d4524.props -preload openie


In [57]:
infobox_str1 = create_baseline_infobox_string(infobox_1, title_1)
print(infobox_str1)

{{Infobox 
| title = V for Vendetta (film)
| label1 = seen | data1 = v
| label2 = united | data2 = regime, nordic supremacist, neo-fascist totalitarian regime, totalitarian regime, supremacist, neo-fascist regime
| label3 = vendetta | data3 = studios
| label4 = reviews | data4 = it
| label5 = uncredited | data5 = alan moore
| label6 = it | data6 = anarchists, people
| label7 = popular | data7 = guy fawkes mask
| label8 = people | data8 = i
}}


In [50]:
def publish_to_sandbox(infobox_str,input_page,credentials):
    input_page = '_'.join(input_page.split(' '))
    with open(credentials,'r') as cred:
        for line in cred:
            if(line[:4] == 'user'): user = line.split('=')[-1].strip('\n')
            if(line[:8] == 'password'): password = line.split('=')[-1]
    site = mwclient.Site('en.wikipedia.org')
    site.login(user, password)
    old_page = site.Pages[input_page]
    old_text = old_page.text()
    old_text = re.sub(r'(?=\{Infobox)(\{([^{}]|(?1))*\})','',old_text,flags = re.S)
#     print(old_page.text())
    page = site.Pages['User:Mohanty-Aniket/sandbox']
    text = infobox_str
    page.save(text + '\n' + old_text,summary='trying baseline-infobox')

In [58]:
# publish_to_sandbox(infobox_str1, title_1, './credentials.txt')